# 锂电池温度预测
锂离子电池材料的主要生产设备是电炉，研究烧结过程的数字化建模，通过电炉空间温度推测产品内部温度，设计烧结过程的温度场和浓度场的最优控制律，搭建产品制备过程运行平台，有望最终实现该过程的效率提升和协同优化，达到提高产品一致性，降低生产能耗的目标。初赛提供了电炉17个温区的实际生产数据，分别是电炉上部17组加热棒设定温度T1-1~T1-17，电炉下部17组加热棒设定温度T2-1~T2-17，底部17组进气口的设定进气流量V1-V17，选手需要根据提供的数据样本构建模型，预测电炉上下部空间17个测温点的测量温度值。初赛考核办法采用测试集各行数据的加热棒上部温度设定值、加热棒下部温度设定值、进气流量3类数据作为输入，选手分别预测上部空间测量温度、下部空间测量温度。将选手预测的上部空间测量温度、下部空间测量温度与测试集数据的测量值进行比较。采用MAE平均绝对误差作为评价指标。

## 环境配置

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import optuna
import optuna.visualization as ov
from joblib import dump, load, Parallel, delayed
import numpy as np

## 数据准备

In [ ]:
# 数据提取
df_train = pd.read_csv("./datasets/train.csv")
df_test = pd.read_csv("./datasets/test.csv")

# 重命名
df_train.columns = (
        ['index', 'datetime'] +
        [f'V{i + 1}' for i in range(17)] + [f'T1-{i + 1}' for i in range(17)] + [f'T2-{i + 1}' for i in range(17)] +
        [f'T1R-{i + 1}' for i in range(17)] + [f'T2R-{i + 1}' for i in range(17)]

)
df_test.columns = df_train.columns[:53]
df_train

## 特征工程

In [ ]:
def generate_features(df, feature_types: [str, ...]):
    df['timestamp'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('timestamp')

    # 时间特征
    if 'time' in feature_types:
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month
        df['day'] = df['timestamp'].dt.day
        df['hour'] = df['timestamp'].dt.hour
        df['minute'] = df['timestamp'].dt.minute
        df['dayofweek'] = df['timestamp'].dt.dayofweek

    # 累积统计特征
    if 'statistical' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_mean'] = df[[f'T1-{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'T1-{i}_std'] = df[[f'T1-{j}' for j in range(1, i + 1)]].std(axis=1)

            df[f'T2-{i}_mean'] = df[[f'T2-{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'T2-{i}_std'] = df[[f'T2-{j}' for j in range(1, i + 1)]].std(axis=1)

            df[f'V{i}_mean'] = df[[f'V{j}' for j in range(1, i + 1)]].mean(axis=1)
            df[f'V{i}_std'] = df[[f'V{j}' for j in range(1, i + 1)]].std(axis=1)

    # 上下温度区总体特征
    if 'overall' in feature_types:
        # Average, min, max, and standard deviation
        df['T1_mean'] = df[[f'T1-{i}' for i in range(1, 18)]].mean(axis=1)
        df['T1_min'] = df[[f'T1-{i}' for i in range(1, 18)]].min(axis=1)
        df['T1_max'] = df[[f'T1-{i}' for i in range(1, 18)]].max(axis=1)
        df['T1_std'] = df[[f'T1-{i}' for i in range(1, 18)]].std(axis=1)

        df['T2_mean'] = df[[f'T2-{i}' for i in range(1, 18)]].mean(axis=1)
        df['T2_min'] = df[[f'T2-{i}' for i in range(1, 18)]].min(axis=1)
        df['T2_max'] = df[[f'T2-{i}' for i in range(1, 18)]].max(axis=1)
        df['T2_std'] = df[[f'T2-{i}' for i in range(1, 18)]].std(axis=1)

        # Difference and ratio between upper and lower heating rods
        df['T1_T2_diff'] = df['T1_mean'] - df['T2_mean']
        df['T1_T2_ratio'] = df['T1_mean'] / df['T2_mean']

    # 上下温度区交互特征
    if 'interaction' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_T2-{i}_diff'] = df[f'T1-{i}'] - df[f'T2-{i}']
            df[f'T1-{i}_T2-{i}_ratio'] = df[f'T1-{i}'] / df[f'T2-{i}']
            df[f'T1-{i}_T2-{i}_interaction'] = df[f'T1-{i}'] * df[f'T2-{i}']

    # 相邻数据点的差值
    if 'difference' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_diff'] = df[f'T1-{i}'] - df[f'T1-{i + 1}']
            df[f'T2-{i}_T2-{i + 1}_diff'] = df[f'T2-{i}'] - df[f'T2-{i + 1}']
            df[f'V{i}_V{i + 1}_diff'] = df[f'V{i}'] - df[f'V{i + 1}']

    # 比例特征
    if 'ratio' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_V{i}_ratio'] = df[f'T1-{i}'] / df[f'V{i}']
            df[f'T2-{i}_V{i}_ratio'] = df[f'T2-{i}'] / df[f'V{i}']
            df[f'T1-{i}_V{i}_interaction'] = df[f'T1-{i}'] * df[f'V{i}']
            df[f'T2-{i}_V{i}_interaction'] = df[f'T2-{i}'] * df[f'V{i}']

    # 滑动窗口特征
    if 'rolling_window' in feature_types:
        window_sizes = [i for i in range(1, 5)]  # Change this list according to your needs
        for window_size in window_sizes:
            for i in range(1, 18):
                df[f'T1-{i}_rolling_mean_{window_size}'] = df[f'T1-{i}'].rolling(window_size).mean()
                df[f'T2-{i}_rolling_mean_{window_size}'] = df[f'T2-{i}'].rolling(window_size).mean()
                df[f'V{i}_rolling_mean_{window_size}'] = df[f'V{i}'].rolling(window_size).mean()

    # 空间梯度特征
    # 假设加热棒之间的空间距离是均匀的
    if 'spatial_gradient' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_gradient'] = (df[f'T1-{i + 1}'] - df[f'T1-{i}']) / i
            df[f'T2-{i}_T2-{i + 1}_gradient'] = (df[f'T2-{i + 1}'] - df[f'T2-{i}']) / i

    # 时间梯度特征
    if 'time_gradient' in feature_types:
        for i in range(1, 18):
            df[f'T1-{i}_time_gradient'] = df[f'T1-{i}'].diff()
            df[f'T2-{i}_time_gradient'] = df[f'T2-{i}'].diff()
            df[f'V{i}_time_gradient'] = df[f'V{i}'].diff()

    # 时空梯度特征
    if 'time_spatial_gradient' in feature_types:
        for i in range(1, 17):
            df[f'T1-{i}_T1-{i + 1}_time_gradient'] = df[f'T1-{i + 1}'].diff() - df[f'T1-{i}'].diff()
            df[f'T2-{i}_T2-{i + 1}_time_gradient'] = df[f'T2-{i + 1}'].diff() - df[f'T2-{i}'].diff()
            df[f'V{i}_V{i + 1}_time_gradient'] = df[f'V{i + 1}'].diff() - df[f'V{i}'].diff()

    return df


# 要使用的特征类型
#
# use_feature_types = ['time', 'statistical', 'overall', 'interaction', 'difference', 'ratio',
#                      'rolling_window',
#                      'spatial_gradient', 'time_gradient', 'time_spatial_gradient']
use_feature_types = ['time']
df_add = generate_features(df_train.copy(), use_feature_types)
df_add_test = generate_features(df_train.copy(), use_feature_types)

df_add.columns

## 训练前预处理

In [ ]:
target_cols = df_add.columns[53: 53 + 17 * 2].tolist()
feature_cols = [col for col in df_add.columns if col not in target_cols + ['index', 'datetime', 'timestamp']]
X = df_add[feature_cols]
X_test = df_add_test[feature_cols]
y = df_add[target_cols]

# 压缩内存
y

## 模型训练

In [ ]:
# 模型训练迭代次数
TRAIN_ROUND = 3
# 超参数训练次数
OPTUNA_ROUND = 50


### 超参数优化

In [ ]:
def objective(trial, x, y_target):
    # 超参数范围
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression_l1',
            'metric': 'mae',
            'min_child_weight': 5,
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'lambda_l2': 10,
            'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'seed': 2023,
            'nthread': 16,
            'verbose': -1,
    }

    # 5-Fold 交叉验证
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    mae_scores_target = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = x.iloc[train_index], x.iloc[val_index]
        y_train, y_val = y_target.iloc[train_index], y_target.iloc[val_index]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        gbm = lgb.train(params, train_data, valid_sets=val_data, num_boost_round=TRAIN_ROUND)

        y_val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        mae = mean_absolute_error(y_val, y_val_pred)

        mae_scores_target.append(mae)

    # 平均mae
    average_mae = np.mean(mae_scores_target)
    trial.report(average_mae, step=0)

    #
    return average_mae


### 多目标训练

In [ ]:

def train_by_multi_targets():
    for i, target in enumerate(y.columns):
        y_target = y[target]

        # 设定超参数目标函数
        study = optuna.create_study(direction='minimize')

        # 超参数调整
        study.optimize(lambda trial: objective(trial, X, y_target), n_trials=OPTUNA_ROUND)

        # 最优超参数
        best_params = study.best_trial.params

        # Train and save a model using the best hyperparameters
        train_data = lgb.Dataset(X, label=y_target)
        gbm_best = lgb.train(best_params, train_data, num_boost_round=TRAIN_ROUND)

        # 保存目标模型
        dump(gbm_best, f'./models/model_{i}.joblib')

        # 打印
        print('Best trial:')
        best_trial = study.best_trial

        print(f'Value: {best_trial.value}')

        print('Params:')
        for key, value in best_trial.params.items():
            print(f'{key}: {value}')

        # 可视化
        ov.plot_optimization_history(study).show()
        ov.plot_param_importances(study).show()


# TODO:并行版本

train_by_multi_targets()

## 模型推理

In [ ]:
def predict_by_multi_targets():
    # Initialize an empty dataframe to store the predictions
    df_predictions = pd.DataFrame()

    # For each target, load the corresponding model and make predictions
    for i in range(len(y.columns)):
        # Load the model
        gbm_best_saved = load(f'model_{i}.joblib')

        # Make predictions
        predictions = gbm_best_saved.predict(X_test)

        # Store the predictions in the dataframe
        df_predictions[f'target_{i}'] = predictions

    # Save the predictions to a csv file
    df_predictions.to_csv('predictions.csv', index=False)
